# ch 17_9 Simple DNN

이번 챕터에서는 이론으로 배운 딥러닝 모델을 간단히 구현해보겠습니다. 이전 펭귄 클래시피케이션 모델을 다층 신경망 방식으로 구현해보겠습니다. pytorch의 도움을 받으면 어렵지않게 구현할 수 있습니다.

## 데이터 셋 준비

## Deep Neural Network

### 모델 작성

### 하이퍼 파라미터 셋팅

### 모델 학습

### 결과 시각화

## 정리

이번 챕터에서는 이론으로만 배웠던 딥러닝을 실제로 구현해보았습니다. 그 결과, 지금까지 우리가 linear regression이나 logistic regression을 Grandient Descent 방식으로 학습시킨 코드와 그다지 다르지 않았습니다. 차이가 있다면 신경망 레이어를 여러 겹 쌓고, 그 사이 사이에 비선형 함수인 sigmoid를 끼워넣은 모델이라는 점입니다. 선형 레이어 사이 사이에 비선형성을 넣어준다는 점을 꼭 기억해주시고 넘어가시기 바랍니다.